In [24]:
import gurobipy as gb
import numpy as np

In [25]:
model = gb.Model()

In [26]:
#dados do problema
items = range(1,6)
time = range(1,11)
s = {1:4.5, 2:6.2, 3:4.2, 4:4.0, 5:9.5}
h = {1:0.9, 2:0.4, 3:0.8, 4:0.5, 5:0.1}
b = {1:0.8, 2:0.6, 3:0.7, 4:0.2, 5:0.5}
sp = {1:9.2, 2:6.7, 3:6.7, 4:7.2, 5:4.3}
Ct = 48
d = {
        (1,1):1, (1,2):6, (1,3):10, (1,4):6, (1,5):5, (1,6):7, (1,7):5, (1,8):9, (1,9):4, (1,10):5,
        (2,1):2, (2,2):9, (2,3):0, (2,4):0, (2,5):0, (2,6):0, (2,7):8, (2,8):8, (2,9):2, (2,10):9,
        (3,1):3, (3,2):4, (3,3):7, (3,4):0, (3,5):2, (3,6):7, (3,7):2, (3,8):4, (3,9):4, (3,10):10,
        (4,1):4, (4,2):4, (4,3):9, (4,4):4, (4,5):8, (4,6):9, (4,7):3, (4,8):4, (4,9):8, (4,10):6,
        (5,1):5, (5,2):8, (5,3):3, (5,4):3, (5,5):10, (5,6):8, (5,7):8, (5,8):7, (5,9):9, (5,10):2
        }
d_tau = {1:0, 2:0, 3:0, 4:0, 5:0}
for i in items:
    for t in time:
        d_tau[i] = d_tau[i] + d[i,t]

In [27]:
m = model

model._I = items
model._T = time

In [28]:
class expando(object):
    pass
variables = expando()
constraints = expando()

In [33]:
#variáveis
m = model

model._I = items
model._T = time

m._x = \
    m.addVars(((i,t) for i in m._I for t in m._T),
    lb = 0.0, ub=gb.GRB.INFINITY,
    vtype = gb.GRB.CONTINUOUS,
    name = "x")

m._Ix = \
    m.addVars(((i,t) for i in m._I for t in m._T),
    lb = 0.0, ub=gb.GRB.INFINITY,
    vtype = gb.GRB.CONTINUOUS,
    name = "stock_x")

m._y = \
    m.addVars(((i,t) for i in m._I for t in m._T),
    lb = 0.0, ub=1.0,
    vtype = gb.GRB.BINARY,
    name = "y")

variables.x = m._x
variables.Ix = m._Ix
variables.y = m._y

#update necessário para incluir as novas variáveis
m.update()

In [30]:
#restrições

x = model._x
Ix = model._Ix
y = model._y

def minimo(a, b):
    if a < b:
        return a
    else:
        return b

def _u_constrs(i,t):
    if t == 1:
        return (Ix[i,t] - x[i,t] + d[i,t] == 0)
    else:
        return (Ix[i,t] - Ix[i,t-1] - x[i,t] + d[i,t] == 0) #sum(b.values())*v[t] + w[i,t] <= 0)

constraints.u_constrs = model.addConstrs(
    (_u_constrs(i, t) for i in model._I for t in model._T),
    name = "u_constrs"
)
print(f"# Original Problem | U constraints: {len(constraints.u_constrs)}")

def _v_constrs(t):
    return (gb.quicksum(sp[i]*y[i,t] + b[i]*x[i,t] for i in model._I) <= Ct)

constraints.v_constrs = model.addConstrs(
    (_v_constrs(t) for t in model._T),
    name = "v_constrs"
)
print(f"# Original Problem | V constraints: {len(constraints.v_constrs)}")

def _w_constrs(i,t):
    return (x[i,t] <= minimo((Ct-sp[i])/b[i],d_tau[i])*y[i,t])

constraints.w_constrs = model.addConstrs(
    (_w_constrs(i, t) for i in model._I for t in model._T),
    name = "w_constrs"
)
print(f"# Original Problem | W constraints: {len(constraints.w_constrs)}")



# Original Problem | U constraints: 50
# Original Problem | V constraints: 10
# Original Problem | W constraints: 50


In [31]:
#objetivo
model.setObjective(
            gb.quicksum(s[i]*model._y[i,t] + h[i]*model._Ix[i,t] for i in model._I for t in model._T),gb.GRB.MINIMIZE)

In [32]:
m.optimize()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 110 rows, 150 columns and 345 nonzeros
Model fingerprint: 0x90d52aed
Variable types: 100 continuous, 50 integer (50 binary)
Coefficient statistics:
  Matrix range     [2e-01, 6e+01]
  Objective range  [1e-01, 1e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+01]
Found heuristic solution: objective 295.0946429
Presolve removed 20 rows and 25 columns
Presolve time: 0.00s
Presolved: 90 rows, 125 columns, 298 nonzeros
Variable types: 80 continuous, 45 integer (45 binary)

Root relaxation: objective 8.206201e+01, 47 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   82.06201    0   33  295.09464   82.06201  72.2%     -    0s
H    0     0                     212.6825000   82.06201  